### This jupyter notebook employs a fully connective neural network(FC) or its alias artificial neural network (ANN) to learn the mapping between input current configuration between output magnetic field 

In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy
import torch
if torch.cuda.device_count():
    device = 'cuda'
    print('Good to go')
else:
    device = 'cpu'
    print('Using cpu')

In [ ]:
from ReadData import ReadFolder 
import glob
import os 

# print(os.getcwd())
foldername="./Data/"
filepattern = "MagneticField[0-9]*.txt"
data = ReadFolder(foldername,filepattern)
fileList = glob.glob(foldername+filepattern)
position = data[:,:,:3].reshape(-1,3)
Bfield = data[:,:,3:].reshape(-1,3)
print(fileList)
print(data.shape)
print('Bfield shape', Bfield.shape)


In [ ]:
from Neural_network import NN_net, Plain_fc_block, weight_init, eMNS_Dataset
from Training_loop import train_part


batch_size = 128
# construct dataset
dataset = eMNS_Dataset(
    train_x=position,
    train_y=Bfield
)

# split the dataset to train, validation, test
train_set, valid_set, test_set = torch.utils.data.random_split(dataset, [0.7,0.2,0.1])

#Using Dataloader for batch train
train_loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_size,shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=valid_set,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_size,shuffle=True)


###############################################
# Config the neural network
###############################################
num_input = 3
num_output = 3
fc_stages = [(num_input,100,1),(100,50,1),(50,25,1)]
fc_network = NN_net(None,fc_stages,None,Plain_fc_block, num_output=num_output)
epochs = 50
learning_rate_decay = .1
learning_rates = [1e-3]
schedule = []
weight_decays = [0]


################################################
# Train the neural network
################################################

for learning_rate in learning_rates:
    for weight_decay in weight_decays:
        fc_network.apply(weight_init)
        optimizer = torch.optim.Adam([{'params':fc_network.parameters()}], lr=learning_rate, weight_decay= weight_decay)
        RMSE_history, RMSE_val_history, loss_history, iter_history = train_part(model=fc_network, optimizer=optimizer, train_loader=train_loader, valid_loader=valid_loader, epochs=epochs, learning_rate_decay=learning_rate_decay, schedule=schedule, weight_decay=weight_decay, verbose=False, device=device)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
ave_site = 5
ave_kernel = 1/ave_site*np.ones(ave_site)
loss_history_conv = np.convolve(loss_history.numpy(),ave_kernel,'same')
plt.title('Train and Val RMSE')
plt.plot(iter_history,RMSE_history,'-o')
plt.plot(iter_history,RMSE_val_history,'-o')
plt.legend(['train acc','val acc'])
plt.xlabel('iterations')
plt.ylabel('RMSE')
plt.show()

plt.title('loss')
plt.plot(iter_history,loss_history,'-o')
plt.plot(iter_history,loss_history_conv,'-*')
plt.legend(['loss','loss_conv'])
plt.xlabel('iterations')
plt.ylabel('loss')
plt.show()